In [1]:
import copy
import numpy as np
import pandas as pd
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
from dask import delayed, compute
from dask.datasets import timeseries
import tc_distributed as tcd
import transforms_distributed as tfd

una_oprs = tfd.unary_operators
bina_oprs = tfd.binary_operators
oprs_list = [una_oprs, bina_oprs]

### 对代码的一系列简化测试

In [2]:
# 导入数据：这里刚读入的数据里都有第一列的一列索引，要把他删除掉
cur_dat = pd.read_csv('result/curr_php0iVrYT.csv')
cur_dat.drop(cur_dat.columns[0], axis=1, inplace=True)
cur_gen = pd.DataFrame(cur_dat.iloc[:, :-1], columns=cur_dat.columns[:-1])
print(cur_gen)
prev_dat = pd.read_csv('result/prev_php0iVrYT.csv')
prev_gen = prev_dat.drop(prev_dat.columns[0], axis=1, inplace=False)
prev_gen

      (V2/V4)_adde  (V1*V3)_cos  (V1+V2)_cos  (V2/V4)_cos  V4_rbfFeature_cos  \
0         3.718282    -0.729947    -0.145500     0.540302           0.939319   
1         2.838971     0.862013     0.004426     0.992726           0.921006   
2         3.025974     0.983987    -0.547729     0.953035           0.952616   
3         2.780782    -0.729947    -0.275163     0.998048           0.969351   
4         2.940504    -0.367460     0.960170     0.975410           0.760382   
...            ...          ...          ...          ...                ...   
1135      2.869797     0.759825     0.753902     0.988543           0.774508   
1136      2.993282    -0.598846     0.907447     0.962425           0.878350   
1137      3.172827     0.154668    -0.911130     0.898461           0.943629   
1138      3.218282    -0.883849    -0.989992     0.877583           0.886204   
1139      3.127373     0.325439     0.004426     0.917483           0.893915   

      (V2-V4)_cos  (V3*V4)_cos  (V1*V4)

,V1,V2,V3,V4,V1_adde,V1_cos,V1_degree,V3_ln,V1_negative,V1_radian,...,V4_rbfFeature,(V2/V4),(V2-V4),(V1-V2),(V1+V2),(V3+V4),(V1+V4),(V1*V3),(V3*V4),(V1*V4)
0,4.0,4.0,1000.0,4.0,6.718282,-0.653644,229.18310,6.907755,-4.0,0.069813,...,0.350156,1.000000,0.0,0.0,8.0,1004.0,8.0,4000.0,4000.0,16.0
1,4.0,7.0,1750.0,58.0,6.718282,-0.653644,229.18310,7.467371,-4.0,0.069813,...,0.400142,0.120690,-51.0,-3.0,11.0,1808.0,62.0,7000.0,101500.0,232.0
2,13.0,8.0,2000.0,26.0,15.718282,0.907447,744.84510,7.600903,-13.0,0.226893,...,-0.309071,0.307692,-18.0,5.0,21.0,2026.0,39.0,26000.0,52000.0,338.0
3,16.0,1.0,250.0,16.0,18.718280,-0.957659,916.73240,5.521461,-16.0,0.279253,...,-0.248219,0.062500,-15.0,15.0,17.0,266.0,32.0,4000.0,4000.0,256.0
4,4.0,2.0,500.0,9.0,6.718282,-0.653644,229.18310,6.214608,-4.0,0.069813,...,-0.706896,0.222222,-7.0,2.0,6.0,509.0,13.0,2000.0,4500.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,2.0,5.0,1250.0,33.0,4.718282,-0.416147,114.59155,7.130899,-2.0,0.034907,...,0.684859,0.151515,-28.0,-3.0,7.0,1283.0,35.0,2500.0,41250.0,66.0
1136,2.0,11.0,2750.0,40.0,4.718282,-0.416147,114.59155,7.919356,-2.0,0.034907,...,-0.498398,0.275000,-29.0,-9.0,13.0,2790.0,42.0,5500.0,110000.0,80.0
1137,4.0,5.0,1250.0,11.0,6.718282,-0.653644,229.18310,7.130899,-4.0,0.069813,...,-0.337369,0.454545,-6.0,-1.0,9.0,1261.0,15.0,5000.0,13750.0,44.0
1138,2.0,1.0,250.0,2.0,4.718282,-0.416147,114.59155,5.521461,-2.0,0.034907,...,-0.481709,0.500000,-1.0,1.0,3.0,252.0,4.0,500.0,500.0,4.0


In [ ]:
# 通过调用操作符生成子代特征
cands_bag = tcd.new_cands(cur_dat, prev_gen)
cands_bag = compute(compute(cands_bag)[0])[0]
cands_bag

In [6]:
%%time
# 2021/1/31改进后完成只用3min 48s
# 生成下一代经历了竞争灭绝环境限制的子特征
next_g = tcd.getNextGen(cur_gen, prev_gen)
next_g.to_csv('test.csv')
next_g

2021-01-31 14:16:42,868 - tc_distributed - INFO - Start to generate children candidates
2021-01-31 14:16:42,878 - tc_distributed - INFO - Finish generating children candidates
2021-01-31 14:16:48,229 - tc_distributed - INFO - Start Data Cleaning after new_cands function
2021-01-31 14:16:48,239 - tc_distributed - INFO - Set type to float32 at first && deal with inf.
2021-01-31 14:16:48,239 - tc_distributed - INFO - Set type to float32 at first && deal with inf.
2021-01-31 14:16:48,239 - tc_distributed - INFO - Set type to float32 at first && deal with inf.
2021-01-31 14:16:48,239 - tc_distributed - INFO - Set type to float32 at first && deal with inf.
2021-01-31 14:16:48,250 - tc_distributed - INFO - Remove columns with half of nan
2021-01-31 14:16:48,250 - tc_distributed - INFO - Set type to float32 at first && deal with inf.
2021-01-31 14:16:48,250 - tc_distributed - INFO - Set type to float32 at first && deal with inf.
2021-01-31 14:16:48,258 - tc_distributed - INFO - Set type to flo

Wall time: 3min 48s


,(V1-V2)_abs,(V1+V2)_adde,(V1*V3)_adde,V1_sin_adde,(V1*V3)_cos,V1_sin_degree,(V1*V3)_degree,(V1+V2)_negative,V1_sin_radian,(V1*V3)_radian,...,(V1_square*(V1+V2)),(V1_adde*V2_tanh),(V2_ln*V1_recip),(V1_square*(V1*V3)),(V1_mmnorm*(V1-V2)),(V1_negative*(V1+V2)),(V4_ln*(V1*V3)),((V2+V4)*V3),(V2_ln*(V1*V3)),((V3-V4)*V1)
0,2.0,8.718282,2002.718262,3.627579,-0.367460,52.098904,1.145916e+05,-6.0,0.015870,34.906586,...,24.0,4.715117,2.772589,8000.0,-0.054054,-12.0,8496.990234,74000.0,2772.588867,1860.0
1,4.0,10.718282,3002.718262,2.438866,-0.975682,-16.009329,1.718873e+05,-8.0,-0.004877,52.359879,...,288.0,8.404664,4.158883,108000.0,0.324324,-48.0,7193.686035,6500.0,2079.441650,2934.0
2,11.0,25.718283,25502.718750,2.438866,-0.953072,-16.009329,1.461042e+06,-23.0,-0.004877,445.058960,...,828.0,8.718282,16.999279,918000.0,-0.891892,-138.0,108336.625000,369750.0,72246.937500,25080.0
3,13.0,19.718283,7502.718262,3.627579,-0.524783,52.098904,4.297183e+05,-17.0,0.015870,130.899689,...,68.0,4.718282,5.416101,30000.0,-0.351351,-34.0,29188.652344,240000.0,20310.376953,7402.0
4,14.0,20.718283,8002.718262,3.627579,0.065645,52.098904,4.583662e+05,-18.0,0.015870,139.626343,...,72.0,4.718282,5.545177,32000.0,-0.378378,-36.0,35155.593750,388000.0,22180.708984,7838.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,6.0,14.718282,6752.718262,3.130400,-0.284211,23.612650,3.867465e+05,-12.0,0.007193,117.809723,...,972.0,11.660333,9.887510,546750.0,0.729730,-108.0,17813.636719,12750.0,7415.632812,6624.0
1136,4.0,20.718283,19252.718750,3.375268,-0.108769,37.642559,1.102944e+06,-18.0,0.011467,335.975891,...,882.0,9.718282,16.785267,943250.0,-0.378378,-126.0,86406.250000,275000.0,46159.484375,18627.0
1137,2.0,8.718282,2002.718262,3.627579,-0.367460,52.098904,1.145916e+05,-6.0,0.015870,34.906586,...,24.0,4.715117,2.772589,8000.0,-0.054054,-12.0,5278.114746,18000.0,2772.588867,1972.0
1138,5.0,11.718282,3502.718262,3.627579,0.964887,52.098904,2.005352e+05,-9.0,0.015870,61.086525,...,36.0,4.718274,3.891820,14000.0,-0.135135,-18.0,12130.076172,68250.0,6810.685547,3436.0


In [ ]:
# 生成某个计算对应的自带特征
lis = []
# opr = oprs_list[0].get('kernelapproxrbf')
# candidates = delayed(opr()._exec)(cur_gen)
opr = oprs_list[1].get('minus')
candidates = delayed(opr()._exec)(cur_gen, prev_gen)
lis.append(candidates)
cands_bag = db.from_sequence(lis, partition_size=None, npartitions=None)
# cands_bag = tcd.new_cands(dat, prev_gen=prev_gen)
cands_bag = compute(compute(cands_bag)[0])[0]
cands_bag

In [ ]:
# 调用写好的函数直接生成子特征
# cands_lis = tcd.new_cands(cur_dat, prev_gen=prev_gen)

cands_bag = []
# cands_bag = tcd.new_cands(dat, prev_gen)
# print(compute(compute(cands_bag)[0])[0])
for oprs in oprs_list:
     for opr in oprs.values():
          candidates = None
          if oprs == oprs_list[0]:
               candidates = delayed(opr()._exec)(cur_gen)
          elif oprs == oprs_list[1]:
               candidates = delayed(opr()._exec)(cur_gen, prev_gen)
          cands_bag.append(candidates)
          # candidates_list.append(candidates.compute())
cands_bag = compute(compute(cands_bag)[0])[0]
print(cands_bag)
# print(compute(candidates_list)[0])

In [ ]:
cands_lis = []
for cand in cands_bag:
    cand = delayed(tcd.clean_dat)(cand)
    cands_lis.append(cand)
cands_lis = compute(cands_lis)[0]
cands_lis

In [ ]:
sorted_lis = []
for cand in cands_lis:
    if cand.empty:
        continue
    cand_dat = copy.deepcopy(cand)
    cand_dat[cur_dat.columns[-1]] = cur_dat.iloc[:, -1]
    sorted_gps = tcd.tribesCompetition(cand_dat)
    # tC 返回的结果是delayed格式的
    sorted_lis.append(sorted_gps)
sorted_lis = compute(sorted_lis)[0]
# sorted_lis = compute(compute(sorted_lis)[0])[0]
sorted_lis

In [ ]:
res_lis = []
# for i in range(len(sorted_lis)):
for i in range(4):
    cand_dat = copy.deepcopy(cands_lis[i-4])
    cand_dat[cur_dat.columns[-1]] = cur_dat.iloc[:, -1]
    cand_d = delayed(tcd.plunge_eliminate)(cand_dat, sorted_lis[i-4])
    res_lis.append(cand_d)
res_lis = compute(res_lis)[0]
res_lis

In [ ]:
# 和上一个cell功能类似,此cell只用于生成某个特定operator下的自带特征
res_lis = []
cand = cands_lis[-3]
cand_dat = copy.deepcopy(cand)
cand_dat[cur_dat.columns[-1]] = cur_dat.iloc[:, -1]
sorted_gps = tcd.tribesCompetition(cand_dat)
# tC 返回的结果是delayed格式的
sorted_gps = compute(sorted_gps)[0]
cand_d = delayed(tcd.plunge_eliminate)(cand_dat, sorted_gps)
res_lis.append(cand_d)

res_lis = compute(res_lis)[0]
res_lis

In [ ]:
from distributed import progress
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, f1_score
cand = cands_lis[-4]
d = delayed(train_test_split)(cand, cur_dat.iloc[:, -1])
X_train, X_test, y_train, y_test = compute(d)[0]
s = tcd.eval(X_train, X_test, y_train, y_test)
s = s.compute()
s

In [ ]:
%%time
from sklearn.model_selection import train_test_split

# r = delayed(train_test_split)(cand, cur_dat.iloc[:, -1])
r = train_test_split(cand, cur_dat.iloc[:, -1])
X_train, X_test, y_train, y_test = compute(r)[0]
# X_train, X_test, y_train, y_test
s = tcd.eval(X_train, X_test, y_train, y_test)[0]
compute(s)[0]

### 其余一些小的测试

In [ ]:
import numpy as np
lis = [0, 1, 2, 3, 4, 5, 6]

ind_lis = ()
for i in range(7):
    if lis[i] % 2:
        ind_lis = ind_lis + (i,)
lis = np.delete(lis, ind_lis).tolist()
lis

In [ ]:
dataset = 'data/php0iVrYT.csv'
dat = pd.read_csv(dataset)
res = pd.DataFrame()
N = 1
ddat = dd.from_pandas(dat, npartitions=N)
for col in dat.columns:
    if (dat[col].values > 0).any():
        res[str(col)+ '_abs'] = ddat[col].map(lambda x: x + np.e)
res

In [ ]:
@delayed
def add(x, y=1):
    return x + y

b = db.from_sequence(range(6), npartitions=2)
b = b.map(add, y=2)
b.compute()[0].compute()

### Setting up a cluster

In [ ]:
"""  """
from dask.distributed import LocalCluster, Client, progress
cluster = LocalCluster(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client = Client(cluster)
# client = Client()
client
# client.ncores()

In [ ]:
client.shutdown()

In [ ]:
%%time
from time import sleep
def inc(x):
    sleep(1)
    return x + 1
def add(x, y):
    sleep(1)
    return x + y
# This takes three seconds to run because we call each function sequentially, one after the other
x1 = inc(1)
y1 = inc(2)
z1 = add(x1, y1)
z1

### Parallelize code with dask.delayed

In [ ]:
%%time
from dask import delayed
# This runs immediately, all it does is build a graph
x = delayed(inc)(1)
y = delayed(inc)(2)
z = delayed(add)(x, y)
# To get the result, call compute. Notice that this runs faster than the original code.
z.compute()

In [ ]:
# Look at the task graph for `z`
import graphviz
z.visualize()
# "dot -c" to be run in admin mode

### Parallelize a for-loop

In [ ]:
%%time
results = []
data = [1, 2, 3, 4, 5, 6, 7, 8]
for x in data:
    y = delayed(inc)(x)
    results.append(y)
total = delayed(sum)(results)
print("Before computing:", total) 
print("After computing :", total.compute())
total.visualize()

In [ ]:
import tc_distributed as tcd
import dask
dataset = 'data/php0iVrYT.csv'
dat = tcd.load_data(dataset, art='C')
lis = tcd.tribesCompetition(dat)
dask.compute(lis)

In [ ]:
from dask import delayed
from dask import compute
def fun(x):
    sums = []
    for i in range(x):
        sums.append(delayed(lambda n: n+1)(i))
    res = delayed(sum)(sums)
    return res.compute()
r = delayed(fun)(5)
# r.compute()
r.visualize()

In [ ]:
%%time
import dask.bag as db
import time
@delayed
def add(x, y):
    time.sleep(1)
    return x+y
lis = []
for i in range(3):
    cand = delayed(lambda x, y: add(x, y))(i+1, i+2)
    lis.append(cand)
print(lis)
# type(compute(lis))
b = db.from_sequence(lis)
# b = compute(compute(b)[0])[0]
b = compute(compute(compute(b)[0])[0])[0]
b
# type(b)
# lis = compute(lis)[0]
# lis

In [ ]:
%%time
import time
def add(x, y):
    time.sleep(1)
    return x+y
lis = []
for i in range(3):
    lis.append(add(i+1, i+2))
lis

In [ ]:
import pandas as pd
import numpy as np
from dask import compute, delayed
import tc_distributed as tcd
import dask.bag as db

dataset = 'data/php0iVrYT.csv'
dat = pd.read_csv(dataset)
print(dat)
seq = []
# res = pd.DataFrame()
d = []
for col in dat.columns:
    seq.append(str(col)+'_abs')
    d.append(delayed()(-dat[col]))
d = compute(d)[0]
d = np.transpose(np.array(d, dtype='float32'))
res = pd.DataFrame(d, columns=seq)
res
# for i in range(len(seq)):
#     res[str(seq[i])+ '_abs'] = res_lis[i]

# b = db.from_sequence(dat_lis)
# b = b.map(abs)
# b.compute()

In [ ]:
""" Often we want to delay only some functions, running a few of them immediately. 
    To delay or not to delay, is usually where we need to be thoughtful when using dask.delayed """
# %%time
def inc(x):
    sleep(1)
    return x + 1, 1

def double(x):
    sleep(1)
    return 2 * x

def is_even(x):
    return not x % 2
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
res = []
for i in range(2):
    for x in data:
        if is_even(x):
            y = delayed(double)(x)
        else:
            # continue
            y = delayed(inc)(x)[0]
        res.append(y)
total = delayed(sum)(res)
total2 = total / 5
%time total2.compute()
total2.visualize()

In [ ]:
from dask import delayed
from dask import compute   #  When having multiple outputs, use the dask.compute
x = delayed(np.arange)(10)
y = x ** 2
min_, max_ = compute(y.min(), y.max())
min_, max_

In [ ]:
%%time
dataset = 'data/php0iVrYT.csv'
df = delayed(pd.read_csv)(dataset)
sums = []
counts = []

by_class = df.groupby('Class')
total = by_class.V1.sum()
count = by_class.V1.count()
sums.append(total)
counts.append(count)
sums, counts = compute(sums, counts)
total_delays = sum(sums)
n_flights = sum(counts)
mean = total_delays / n_flights
mean

### 用装饰器完成delayed功能

In [ ]:
from dask import delayed
@delayed
def inc(x):
    return x + 1
@delayed
def add(x, y):
    return x + y

x = inc(15)
y = inc(30)
total = add(x, y)
print("total: ", total.compute())
# total.visualize()
dict(total.dask)

### Dask Distributed, Advanced

In [ ]:
def inc(x):
    return x + 1
# fut = client.submit(inc)
fut = client.submit(inc, 1)
fut

In [ ]:
fut

In [ ]:
from dask.distributed import wait, progress
progress(fut)
wait(fut)

In [ ]:
# grab the information back - this blocks if fut is not ready
client.gather(fut)
# equivalent action when only considering a single future
# fut.result()

In [ ]:
""" Here we see an alternative way to execute work on the cluster: when you submit or map with the inputs as futures, the computation moves to the data rather than the other way around, and the client, in the local Python session, need never see the intermediate values. This is similar to building the graph using delayed, and indeed, delayed can be used in conjunction with futures. """
# Some trivial work that takes time
# repeated from the Distributed chapter.

from dask import delayed
import time

def inc(x):
    time.sleep(5)
    return x + 1

def dec(x):
    time.sleep(3)
    return x - 1

def add(x, y):
    time.sleep(7)
    return x + y

In [ ]:
x = delayed(inc)(1)
y = delayed(dec)(2)
total = delayed(add)(x, y)
# notice the difference from total.compute()
# notice that this cell completes immediately
fut = client.compute(total)
fut

In [ ]:
client.gather(fut) # waits until result is ready

In [ ]:
x = client.submit(inc, 1)
y = client.submit(dec, 2)
total = client.submit(add, x, y)

print(total)     # This is still a future
total = client.persist(total)
client.gather(total)   # This blocks until the computation has finished

### background execution

In [ ]:
def add(x, y):
    return x + y

# Sometimes we defer computations with strings
x = 15
y = 30
z = "add(x, y)"
eval(z)



In [ ]:
def gen():
    res = 20
    yield res
    res += y
    yield res

next(gen())

### dask.bag = map, filter, toolz + parallel execute

In [ ]:
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(10)

In [ ]:
""" dask.bag manipulation """
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
# Operations on Bag objects create new bags.
c = b.filter(is_even).map(lambda x: x ** 2)
c
""" Call the .compute() method to trigger execution, as we saw for Delayed objects. """
c.compute()

In [ ]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

In [ ]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()


In [ ]:
# 同样功能
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

### dask.array

In [ ]:

x = da.random.normal(10, 0.1, size=(20000, 20000),   # 400 million element array 
                              chunks=(1000, 1000))   # Cut into 1000x1000 sized chunks
x

In [ ]:
y = x.mean(axis=0)[::100]
print(x.nbytes/1e9, 'Gigabytes of the input processed lazily')
# y
%time y.compute()     # Time to compute the result
""" Notice that the Dask array computation ran in 4 seconds, but used 29.4 seconds of user CPU time. 
    The numpy computation ran in 19.7 seconds and used 19.6 seconds of user CPU time.
    Dask finished faster, but used more total CPU time 
    because Dask was able to transparently parallelize the computation because of the chunk size. 
"""

In [ ]:
dataset = 'data/php0iVrYT.csv'
df = pd.read_csv(dataset)
a = [da.from_array(df[col], chunks=(374,)) for col in df.columns]
a

In [ ]:
ddf = dd.read_csv(dataset)
col = df.columns[0]
# ddf.V1.nunique().compute()
ddf[col].nunique().compute()

In [ ]:
""" Stack this list of dask.array objects into a single dask.array object with da.stack """
aa = da.stack(a)
aa

### dask.dataframe

In [ ]:
dataset = 'data/php0iVrYT.csv'
ddf = dd.read_csv(dataset)
print(ddf)
ddf.head()

In [ ]:
""" Web Interface """
ddf = client.persist(ddf)
progress(ddf)

print(len(ddf))
ddf.count().compute()

In [ ]:
print((ddf.V1 == 0).sum().compute())
print(ddf.V1.sum().compute())
ddf.V1.mean().compute()

In [ ]:
ddf.groupby('Class').V1.mean().compute()

In [ ]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()

In [ ]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

In [ ]:
print(ddf.info())
sub_V2 = ddf[ddf.V2 >= 25]
sub_V2.shape[0].compute()

In [ ]:
cols = ['V1', 'V3', 'Class']
new_ddf = ddf[cols]
new_ddf = client.persist(new_ddf)
progress(new_ddf)
new_ddf.head()

In [ ]:
import multiprocessing
cpu_count()

In [ ]:
dataset = 'data/php0iVrYT.csv'
df = pd.read_csv(dataset)
a = [da.from_array(df[col], chunks=(374,)) for col in df.columns]
""" Stack this list of dask.array objects into a single dask.array object with da.stack """
aa = da.stack(a)
print(aa)

In [ ]:
N = 5
ddf = dd.from_pandas(df, npartitions=N)
""" Subset dask dataframe by column position """
cols = list(df.columns[2:4])
ddf[cols].compute()